# Qutrit Circuit Transpilation Example 
### Measurement Random Compiling
This notebook demonstrates how to use randomized compiling and circuit decomposition for a simple qutrit (3-level quantum system) measurement circuit, and then transpile the resulting circuits into a native gate set using qcal and cirq.

You can install qcal from https://github.com/akelhashim/qcal

In [ ]:
from qdit.compiling import CircuitDecomposer, RandomCompiling
from qcal.interface.superstaq.transpiler import CirqTranspiler, transpilation_error
from qcal.gate.single_qubit import Rz, X90, Id
from collections import defaultdict
import cirq

In [2]:
# Measurement Circuit:
# A simple measurement circuit is created for a single qutrit (dimension 3) using Cirq.
measurement_circuit = cirq.Circuit()
qutrit = cirq.LineQid(0, dimension = 3)
measurement_circuit.append(cirq.measure(qutrit, key="x"))

# Randomized Compiling Loop:
# Specified a number of cycles.
cycles = 100
circuits = []
corrections = []

for _ in range(cycles):
    # The measurement circuit is compiled with randomization (measurement_twirl=True).
    compiler = RandomCompiling(measurement_circuit, d=3, measurement_twirl=True)
    compiled_circuit = compiler.compile()

    # The compiled circuit is decomposed into a native gate set (RzRx).
    decomposer = CircuitDecomposer(compiled_circuit)
    decomposed_circuit = decomposer.decompose_circuit(num_qudits=1, dim=3, native_gates='RzRx')
    # The measurement operation is appended again to the decomposed circuit.
    decomposed_circuit.append(cirq.measure(qutrit, key="x"))

    # Measurement corrections (needed due to randomization) are collected.
    corrections.append(compiler.get_measurement_corrections())
    # Both the decomposed circuit and its corrections are stored.
    circuits.append(decomposed_circuit)

# The circuits and their corresponding corrections are zipped together for further analysis or execution.
post_procesing = zip(circuits, corrections)

In [3]:
# A gate_mapper dictionary maps gate names to their corresponding native gate implementations.
gate_mapper = defaultdict(lambda: transpilation_error,
                {'Id': Id,
                 'Rz': Rz,
                 'X90': X90,
                }
            )

# A CirqTranspiler is initialized with this mapping.
transpiler = CirqTranspiler(gate_mapper)

# All decomposed circuits are transpiled into the target gate set.
transpiled_circuits = transpiler.transpile(circuits=circuits)

In [4]:
# Some transpiled circuits are printed for review.
print(transpiled_circuits[0:10])

                                             circuit
0  (Cycle X90:(0,), Cycle X90:(0,), Cycle X90:(0,...
1    (Cycle Rz:(0,), Cycle Rz:(0,), Cycle Meas:(0,))
2  (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
3  (Cycle X90:(0,), Cycle X90:(0,), Cycle X90:(0,...
4  (Cycle Rz:(0,), Cycle Rz:(0,), Cycle X90:(0,),...
5  (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
6  (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
7  (Cycle X90:(0,), Cycle X90:(0,), Cycle X90:(0,...
8  (Cycle Rz:(0,), Cycle X90:(0,), Cycle X90:(0,)...
9  (Cycle Rz:(0,), Cycle Rz:(0,), Cycle X90:(0,),...
